# Overview

The purpose of this notebook is to obtain the data needed to calculate the intraday-correlation feature for the Supervised Model 2. This is one of the trickiest features to obtain, as common sources only have short histories of intraday data (Bloomberg typically houses about 6months worth of intraday data). However, for the purpose of the model, several years are needed. 

We source the intraday data from the TAQ database of Wharton Research Data Services (WRDS), and the code requires a login credentials. 

The code proceeds in two steps. The first one downloads to separate csv files the relevant intraday trades for several years (one at a time, given query size) while at the same time resampling it to a 1-minute frequency. The second step loads all of these csv files, merges them together and calculates a continues time series of mean intraday correlation, which is outputed as a separate csv file to be used as a feature.

#Setup

## Load Libraries

In [1]:
import pandas as pd

## Download SPY/IEF Data from WRDS (Optional)

This section is only required once to generate raw csv collection. After this has been collected (right now available in the repository), the code can be run using those files. 

In [ ]:
username = 'rerices'

In [ ]:
import wrds
db = wrds.Connection(wrds_username = username)

Loading library list...
Done


In [ ]:
# list of years for which data will be downloaded
years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

for year in years: #iterates through yearly libraries
    library = 'taqm_' + year
    spy_yearly_prices = pd.DataFrame()
    
    tables = [l for l in db.list_tables(library) if 'ctm_' in l]
    dates = [s[4:] for s in tables if len(s) == 12] #filters out tables which do not correspond to a particular day
    
    for date in dates: #iterates through daily tables to query for intraday trades
        print(date)
        aux = db.raw_sql(f"SELECT time_m, price FROM {library}.ctm_{date} WHERE sym_root = 'SPY' AND time_m > '09:30:00' AND time_m < '16:00:00'")
        aux['time_m'] = pd.to_datetime(aux.time_m, format = '%H:%M:%S.%f', errors = 'coerce')

        # if a different frequency is desired for computation, next line is the place to change it
        aux_reduced = aux.groupby([aux.time_m.dt.hour, aux.time_m.dt.minute, aux.time_m.dt.second]).price.median()
        aux_reduced.index.names = ['hour', 'minute', 'second']
        aux_reduced.name = date

        spy_yearly_prices = pd.concat([spy_yearly_prices, aux_reduced], axis = 1)

    spy_yearly_prices.to_csv(f'spy_yearly_{year}.csv')

In [ ]:
years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

for year in years:  #iterates through yearly libraries
    library = 'taqm_' + year
    ief_yearly_prices = pd.DataFrame()
    
    tables = [l for l in db.list_tables(library) if 'ctm_' in l]
    dates = [s[4:] for s in tables if len(s) == 12] #filters out tables which do not correspond to a particular day
    
    for date in dates:
        print(date)
        aux = db.raw_sql(f"SELECT time_m, price FROM {library}.ctm_{date} WHERE sym_root = 'IEF' AND time_m > '09:30:00' AND time_m < '16:00:00'")
        aux['time_m'] = pd.to_datetime(aux.time_m, format = '%H:%M:%S.%f', errors = 'coerce')

        # if a different frequency is desired for computation, next line is the place to change it
        aux_reduced = aux.groupby([aux.time_m.dt.hour, aux.time_m.dt.minute]).price.mean()
        aux_reduced.index.names = ['hour', 'minute']
        aux_reduced.name = date

        ief_yearly_prices = pd.concat([ief_yearly_prices, aux_reduced], axis = 1)

    ief_yearly_prices.to_csv(f'ief_yearly_{year}.csv')

## Load Data from Stored *CSVs*

In [7]:
filepath = 'PATH_TO_DATA'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

iefs = []
for year in years:
    ief = pd.read_csv(filepath + f'ief_yearly_{year}.csv')
    ief = ief.set_index(['Unnamed: 0', 'Unnamed: 1'])
    iefs.append(ief)
    
iefs = pd.concat(iefs, axis = 1)
iefs = iefs.ffill()

In [10]:
years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020']

spys = []
for year in years:
    spy = pd.read_csv(filepath + f'spy_yearly_{year}.csv')
    spy = spy.set_index(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2'])
    spys.append(spy)
    
spys = pd.concat(spys, axis = 1)
spys = spys.groupby(level = [0, 1]).mean().ffill()

## Calculate Intraday Correlation

In [11]:
intraday_p = pd.Series()
for date in spys.columns:
    intraday_p[date] = pd.concat([spys[date], iefs[date]], axis = 1).pct_change().corr().values[0,1]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


## Save Feature Data to CSV

In [ ]:
savepath = '/content/drive/Shareddrives/Capstone Project/Data/data_model2/'

intraday_p.to_csv('intraday.csv')